SQL 语法

In [ ]:
CREATE TABLE mytable (
  id INT NOT NULL AUTO_INCREMENT,
  -- int 类型，不可为空，默认值为 1，不为空
  col1 INT NOT NULL DEFAULT 1,
  -- 变长字符串类型，最长为 45 个字符，可以为空
  col2 VARCHAR(45) NULL,
  -- 日期类型，可为空
  col3 DATE NULL,
  -- 设置主键为 id
  PRIMARY KEY (`id`));

In [ ]:
-- 返回第 3 ~ 5 行
SELECT *
FROM mytable
LIMIT 2, 3;

In [ ]:
SELECT *
FROM mytable
ORDER BY col1 DESC, col2 ASC;

In [ ]:
-- % 匹配 >=0 个任意字符；

-- _ 匹配 ==1 个任意字符；

-- [ ] 可以匹配集合内的字符，例如 [ab] 将匹配字符 a 或者 b。用脱字符 ^ 可以对其进行否定，也就是不匹配集合内的字符。

-- 使用 Like 来进行通配符匹配。

In [ ]:
SELECT *
FROM mytable
WHERE col LIKE '[^AB]%'; -- 不以 A 和 B 开头的任意文本

In [ ]:
-- WHERE 过滤行，HAVING 过滤分组，行过滤应当先于分组过滤
SELECT col, COUNT(*) AS num
FROM mytable
WHERE col > 2
GROUP BY col
HAVING num >= 2;

In [ ]:
SELECT A.value, B.value
FROM tablea AS A INNER JOIN tableb AS B
ON A.key = B.key;

In [ ]:
-- 外连接保留了没有关联的那些行。分为左外连接，右外连接以及全外连接，左外连接就是保留左表没有关联的行。
-- 检索所有顾客的订单信息，包括还没有订单信息的顾客。
SELECT Customers.cust_id, Customers.cust_name, Orders.order_id
FROM Customers LEFT JOIN Orders
ON Customers.cust_id = Orders.cust_id;

In [ ]:
-- 视图是虚拟的表，本身不包含数据，也就不能对其进行索引操作。
-- 对视图的操作和对普通表的操作一样。

-- 视图具有如下好处：

-- 简化复杂的 SQL 操作，比如复杂的连接；
-- 只使用实际表的一部分数据；
-- 通过只给用户访问视图的权限，保证数据的安全性；
-- 更改数据格式和表示。

In [ ]:
-- 使用存储过程的好处：

-- 代码封装，保证了一定的安全性；
-- 代码复用；
-- 由于是预先编译，因此具有很高的性能。

In [ ]:
-- 存储过程更像是完成一个任务或一系列复杂操作的工具
CREATE PROCEDURE add_numbers( IN a INT, IN b INT, OUT sum_result INT )
BEGIN
    SET sum_result = a + b;
END;

-- 调用存储过程
CALL add_numbers(5, 10, @sum_result);
SELECT @sum_result;

In [ ]:
-- 命令行中创建存储过程需要自定义分隔符，因为命令行是以 ; 为结束符，而存储过程中也包含了分号，因此会错误把这部分分号当成是结束符，造成语法错误。
delimiter //
//
 delimiter ;

In [ ]:
-- 在存储过程中使用游标可以对一个结果集进行移动遍历。
-- 游标主要用于交互式应用，其中用户需要对数据集中的任意行进行浏览和修改。

-- 使用游标的四个步骤：

-- 声明游标，这个过程没有实际检索出数据；
-- 打开游标；
-- 取出数据；
-- 关闭游标；
delimiter //
create procedure myprocedure(out ret int)
    begin
        declare done boolean default 0;

        declare mycursor cursor for
        select col1 from mytable;
        -- 定义了一个 continue handler，当 sqlstate '02000' 这个条件出现时，会执行 set done = 1
        declare continue handler for sqlstate '02000' set done = 1;

        open mycursor;

        repeat
            fetch mycursor into ret;
            select ret;
        until done end repeat;

        close mycursor;
    end //
 delimiter ;

In [ ]:
-- 触发器会在某个表执行以下语句时而自动执行：DELETE、INSERT、UPDATE。
-- 触发器必须指定在语句执行之前还是之后自动执行，之前执行使用 BEFORE 关键字，之后执行使用 AFTER 关键字。
-- BEFORE 用于数据验证和净化，AFTER 用于审计跟踪，将修改记录到另外一张表中。
CREATE TRIGGER mytrigger AFTER INSERT ON mytable
FOR EACH ROW SELECT NEW.col into @result;